In [1]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
 # 데이터셋 로드
iris = load_iris()
X = iris.data[:,2:] # 꽃잎의 길이, 너비
Y = iris.target
#shuffle을 주지 않으면 y데이터가 sort가 자동으로 되어있기때문에   000000,1111,2222로 되어있어 섞어줘야함
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)

In [3]:
 # 약한 학습기 구축
log_model = LogisticRegression()
rnd_model = RandomForestClassifier()
svm_model = SVC()

In [4]:
# 앙상블 모델 구축
# 만약에 모든 모델이 predict_proba() 메서드가 있으면, 예측의 평균을 내어 soft voting(간접 투표)도 할수 있다.
# 간접 투표 방식은 확률이 높은 투표에 비중을 두기 때문에 성능이 더 높다. (voting='soft' 사용)
# svc는 기본적으로 predict_proba를 제공하지 않아, probability = True 지정 해야 사용 가능
# 대신 svc에서 probability = True를 지정하면 교차 검증을 사용해서 확률을 추정하기 때문에 훈련 속도 느려짐
# 대신 성능을 올라감
voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',rnd_model),('svc',svm_model)], # 3개의 약한 학습기
    voting='hard' # 직접 투표(hard voting)
)

In [5]:
 # 앙상블 모델 학습
voting_model.fit(x_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [6]:
 # 모델 비교 위에꺼랑 별개로 다시 학습 시키는 것
for model in (log_model,rnd_model,svm_model,voting_model):
  model.fit(x_train,y_train)
  y_pred = model.predict(x_test)
  print(model.__class__.__name__," : ",accuracy_score(y_test,y_pred))
#모델 클래스에 있는 메타데이터 이름값을 가져옮
#클래스에서 메타데이터를 호출해 추려내는 개념

LogisticRegression  :  1.0
RandomForestClassifier  :  0.9555555555555556
SVC  :  1.0
VotingClassifier  :  1.0


2.앙상블 모델2 - 사이킷런의 배깅과 페이스팅

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


In [8]:
 # 모델 구축
# BaggingClassifier에서 사용한 분류기가 클래스 확률추정(predict_proba)이 가능하면 자동으로 간접 투표 사용
bag_model = BaggingClassifier(
    DecisionTreeClassifier(), # 약한 학습기(결정 트리)
    n_estimators=500, # 약한 학습기(결정 트리) 500개 생성 / 랜덤포레스트와 유사함
    max_samples=0.05, # 0.0~1.0 사이 실수 선택가능함 (실수 x 샘플 수) 혹은 샘플수 지정
    bootstrap=True, # True : 배깅, False : 페이스팅 / 중복을 허용하겠다 안하겠다라는 뜻
    n_jobs=-1 # 훈련과 예측에 사용할 CPU 코어 수 결정(-1 : 가용한 모든 코어 사용)
)

In [9]:
 # 데이터셋 로드
iris = load_iris()
X = iris.data[:,2:] # 꽃잎의 길이, 너비
Y = iris.target
#shuffle을 주지 않으면 y데이터가 sort가 자동으로 되어있기때문에   000000,1111,2222로 되어있어 섞어줘야함
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)

In [10]:
 # 모델 학습
bag_model.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.05,
                  n_estimators=500, n_jobs=-1)

In [11]:
 # 모델 예측
y_pred = bag_model.predict(x_test)

# 모델 평가
print(bag_model.__class__.__name__," : ",accuracy_score(y_test,y_pred))

BaggingClassifier  :  0.9555555555555556


oob 평가
* 배깅(중복 허용 샘플링)을 하다보면 평균적으로 훈련 샘플의 약 63%정도만 추출되고 나머지 약 37%는 추출되지 않고, 이렇게 추출되지 않은 샘플들을 oob(out-of-bag)샘플이라고 부른다.
* 예측기가 훈련되는 동안에는 oob샘플을 사용하지 않으므로, 검증 세트나 교차 검증을 사용하지 않고 oob샘플만을 가지고 모델 최적화를 위한 평가를 할 수 있다.
* 앙상블의 평가는 각 예측기의 oob평가의 평균으로 확인한다.

In [12]:
 # 데이터셋 로드
iris = load_iris()
X = iris.data[:,2:] # 꽃잎의 길이, 너비
Y = iris.target
#shuffle을 주지 않으면 y데이터가 sort가 자동으로 되어있기때문에   000000,1111,2222로 되어있어 섞어줘야함
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)

In [13]:
 # 모델 구축
bag_model = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    bootstrap = True,
    n_jobs = -1,
    oob_score = True # oob평가를 위해 True를 지정한다.
)

# 모델 학습
bag_model.fit(x_train,y_train)

# 모델 평가(oob_score_)
print('oob_score : ',bag_model.oob_score_)

# 모델 평가
y_pred = bag_model.predict(x_test)
print('test_score : ',accuracy_score(y_test,y_pred))

oob_score :  0.9523809523809523
test_score :  0.9333333333333333


# 랜덤 패치와 랜덤 서브스페이스
* 위에서는 훈련 샘플을 랜덤 샘플링하여 각 예측기의 오차 다양성을 주었지만, 이번에는 훈련 데이터들의 입력 특성들을 무작위로 샘플링하여 예측기를 만들어서 예측기에 대한 오차 다양성을 줄 수 있다.
* 훈련 데이터들의 입력 특성들을 무작위로 샘플링하는 것을 랜덤 패치 방식과 랜덤 서브스페이스 방식이라고 한다.
* 랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링하는 방식
  - ex) (bootstrap=True or False, max_samples<1.0, bootstrap_features=True, max_features<1.0)
* 랜덤 서브스페이스 방식 : 훈련 샘플은 모두 사용하고, 특성만 샘플링 하는 것
  - ex) (bootstrap=False, max_samples=1.0, bootstrap_features=True, max_features<1.0)

#4. 랜덤포레스트

* 랜덤포레스트는 일반적으로 배깅방법을 사용한 결정트리 앙상블 모델이다.
* 그래서 BaggingClassifier에 DecisionTreeClassifier를 넣는 대신, RandomForestClassifier를 사용할 수 있다.
* 그래서 RandomForestClassifier는 DecisionTreeClassifier와 BaggingClassifier 매개변수 모두 가지고 있다.
* 랜덤포레스트 모델은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 것이 아니라, 무작위로 선택한 특성들 중에서 최선의 특성을 찾는 방식을 채택하여 무작위성을 더 가지게 된다.
* 이를 통해 약간의 편향은 손해보지만, 더욱 다양한 트리를 만들므로 분산을 전체적으로 낮추어서 더 훌륭한 모델을 만들 수 있다.

BaggingClassifier에 DecisionTreeClassifier를 넣는 대신, RandomForestClassifier를 사용

In [14]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 모델 구축
rnd_model = RandomForestClassifier(
    n_estimators = 500, # 예측기 500개
    max_leaf_nodes = 16, # 자식노드의 최대 개수 / 결정트리의 개념이 들어가 있기때문에 사용가능
    n_jobs = -1 # CPU 코어 구동 개수
)

# 모델 학습
rnd_model.fit(x_train,y_train)

# 모델 예측
y_pred_rf = rnd_model.predict(x_test)

# 모델 평가
print("rnd_model : ",accuracy_score(y_pred_rf,y_test))

rnd_model :  0.9555555555555556


# 1. 엑스트라 트리

* 랜덤포레스트는 앞에서 말한 것 처럼 각 노드에서 무작위로 특성을 뽑은 다음 최적의 특성과 임계값을 선택한다.
* 하지만 엑스트라 트리는 최적의 특성과 임계값을 찾는것 대신, 후보 특성을 사용해 무작위로 분할한 다음에 최상을 분할을 선택한다.
* 이렇게되면 기본적으로 편향이 많은 랜던포레스트보다 더욱 편향이 심해지지만, 분산을 더욱 낮출 수 있게 된다.
* 트리 알고리즘에서는 모든 노드에서 최적의 특성과 임계값을 고르는데 시간이 많이 들지만, 엑스트라 트리를 사용하면 훈련과 예측속도가 빨라진다.
* 엑스트라 트리는 ExtraTreesClassifier를 이용하면 사용할 수 있다.
* RandomForestClassifier와 ExtraTreesClassifier 중 어떤 것이 더 좋을지는 판단하기 어렵기 때문에, 교차검증을 통해서 서로 비교해보고, 더 나은 모델을 선택하여 그리드 탐색방법을 사용해 하이퍼파라미터 튜닝을 한다.


# 2. 특성 중요도

* 랜덤포레스트는 성능이 좋다는 장점말고, 특성의 상대적 중요도를 측정하기 쉽다.(트리기반 모델은 특성 중요도 제공)
* 사이킷런에서는 어떤 특성을 사용한 노드가 평균적으로 불순도를 감소시키는지 확인하여 특성 중요도를 측정하고, 훈련이 끝나고 난 뒤에 특성마다 자동으로 점수를 계산하고 저장한다.
* 저장된 값은 featureimportances 변수에 저장되어 있다.


In [15]:
 # 데이터셋 정의
x = iris.data[:,:]
y = iris.target

# 모델 구축
rnd_model = RandomForestClassifier(
    n_estimators = 500,
    n_jobs = -1
)

# 모델 학습
rnd_model.fit(x,y)

# 특성 중요도 확인 (전체 특성 중요도 합 : 1)
for feature_name,feature_imp in zip(iris['feature_names'],rnd_model.feature_importances_):
  print(feature_name,' : ',feature_imp)

sepal length (cm)  :  0.08650721186734646
sepal width (cm)  :  0.020414513973320135
petal length (cm)  :  0.46027478888419576
petal width (cm)  :  0.43280348527513757


5.부스팅
부스팅이란, 약한 학습기를 여러 개들을 서로 연결하고 보완해가면서 더욱 강한 학습기를 만드는 앙상블 방법이다

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# 아다부스트 모델 구축
# 아다부스트의 학습기 : Decision Tree (max_depth =1) 사용
# 학습기 개수(n_estimators) : 200개
# SAMME(Stagewise Additive Modeling using a Multiclass Exponential loss function) 알고리즘 사용
# 기본 학습기가 확률 추정(predict_proba)이 가능하면 SAMME.R 사용 -> 일반적으로 성능이 더 좋음
ada_model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators = 200,
    algorithm = 'SAMME.R',
    learning_rate=0.5
)

# 모델 학습
ada_model.fit(x,y)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [17]:
 # 데이터셋 로드
iris = load_iris()
X = iris.data[:,:]  # 꽃잎의
Y = iris.target
#shuffle을 주지 않으면 y데이터가 sort가 자동으로 되어있기때문에   000000,1111,2222로 되어있어 섞어줘야함
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)

In [18]:
from sklearn.tree import DecisionTreeRegressor

# 결정트리(max_depth=3) 모델 구축 및 학습
tree_reg_model_1 = DecisionTreeRegressor(max_depth=3)
tree_reg_model_1.fit(x,y)

# 첫 번째 학습기에서 발생한 잔차를 목적함수로 모델 학습
residual_1 = y - tree_reg_model_1.predict(x)
tree_reg_model_2 = DecisionTreeRegressor(max_depth=3)
tree_reg_model_2.fit(x,residual_1)

# 두 번째 학습기에서 발생한 잔차를 목적함수로 모델 학습
residual_2 = y - tree_reg_model_2.predict(x)
tree_reg_model_3 = DecisionTreeRegressor(max_depth=3)
tree_reg_model_3.fit(x,residual_2)

# 새로운 데이터를 세 개의 트리를 포함한 앙상블 모델로 예측
x_new = [[1.4,0.2, 3, 0.5]]
prediction = sum(tree.predict(x_new) for tree in [tree_reg_model_1,tree_reg_model_2,tree_reg_model_3])
prediction
#예측 한것의 보정에 보정을함

array([1.02083333])

In [19]:
# 사이킷런에서 제공하는 GBRT 앙상블을 다음과 같이 간단하게 훈련시킬 수 있다.
from sklearn.ensemble import GradientBoostingRegressor

# GBRT 모형 구축
# GBRT 앙상블 모형도 마찬가지로 n_estimators, max_depth, min_samples_leaf 등을 통해 모델 규제가 가능하다.
# 추가적으로 learning_rate가 각 트리의 기여 정도를 조절한다.
# learning_rate가 0.1보다 낮게 설정되면 훈련을 위한 트리가 더 많이 필요하지만 성능은 좋아진다.
# 이러한 방식을 축소(shrinkage)라고 부르는 규제 방법이다.
gbrt = GradientBoostingRegressor(max_depth = 3,
                                 n_estimators = 3,
                                 learning_rate = 1)

# GBRT 모형 학습
gbrt.fit(x,y)

GradientBoostingRegressor(learning_rate=1, n_estimators=3)

In [20]:
 # 최적의 estimators수를 찾기 위한 간단한 방법
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# train set과 validation set을 8:2로 분리
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2)

# GBRT(max_depth=3,예측기 수=120) 모형 구축
gbrt = GradientBoostingRegressor(max_depth=3, n_estimators=120) #120개 최선인지 알수 없음

# GBRT 모형 학습
gbrt.fit(x_train,y_train)

# staged_predict를 활용하여 훈련 각 단계에서 앙상블에 의해 만들어진 예측을 반복자로 반환
#예측기를 1개 썻을때 2개썻을때 계속 리스트로 만들어줌
errors = [mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(x_val)]

# validation 검증 결과에서 가장 좋은 성능을 보인 예측기 수를 추출
bst_estimators_num = np.argmin(errors) #error최소값

# 최고의 일반화 성능을 가진 하이퍼파라미터(n_estimators)를 가지고 재 모델 구축
print("best_est_num : ",bst_estimators_num)
gbrt_best = GradientBoostingRegressor(max_depth=3,n_estimators=bst_estimators_num)

# 데이터셋(train + valid)를 가지고 학습
gbrt_best.fit(x,y)

best_est_num :  32


GradientBoostingRegressor(n_estimators=32)

조기종료(early stopping)를 활용하는 방법

In [21]:
 # 과대적합을 피하는 것이 가장 중요함
# validation set으로 학습이 더이상 안되는 구간을 찾아서 조기종료
# warm_start : fit 메서드 호출될 때마다 기존 트리 유지 및 훈련 추가할 수 있게 하는 하이퍼 파라미터
# subsample : 각 트리는 무작위로 선택된 25% 훈련 샘플로 학습(배깅의 한 방법) => 편향 상승 => 분산 감소 => 훈련 속도 상승
# 위 subsample 방법을 "확률적 그래디언트 부스팅"이라 부른다.
gbrt = GradientBoostingRegressor(max_depth=3, warm_start=True,subsample=0.25)


min_val_error = float('inf') #무한대inf
error_going_up = 0
best_estimator = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators #모형을 구축한것에 변수를 할당
  gbrt.fit(x_train,y_train)
  y_pred = gbrt.predict(x_val)
  val_error = mean_squared_error(y_val, y_pred) #mse 반복 출력
  if val_error < min_val_error:  #최적의 estimater를 선정
    min_val_error = val_error
    best_estimator = n_estimators
    error_going_up = 0
  else:
    error_going_up += 1 # 성능 향상이 되지 않을 때마다 +1
    if error_going_up == 5:
      break # 성능 향상 연속 5회 : 조기 종료

In [22]:
error_going_up

5

In [23]:
best_estimator

17